# Submit Molecule on Slab Geometry Optimization

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm import load_node
from aiida.orm.code import Code
from aiida.orm import Code, Computer
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.data.singlefile import SinglefileData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.base import Int
from aiida.work.workfunction import workfunction

from ase.data import covalent_radii
from ase.neighborlist import NeighborList
import ase.neighborlist

from IPython.display import display, clear_output
import ipywidgets as ipw
import numpy as np
from numpy.linalg import norm
import scipy.stats
import nglview
import tempfile
import shutil
from os import path

from structure_browser import StructureBrowser
from collections import OrderedDict

In [ ]:
# from aiida import load_dbenv, is_dbenv_loaded
# from aiida.backends import settings
# if not is_dbenv_loaded():
#     load_dbenv(profile=settings.AIIDADB_PROFILE)

# from aiida.orm.querybuilder import QueryBuilder
# from aiida.orm.data.structure import StructureData
# from aiida.orm.data.parameter import ParameterData
# from aiida.orm.code import Code
# from aiida.orm.computer import Computer
# from aiida.orm import load_node 

# import ipywidgets as ipw
# from IPython.display import display, clear_output, HTML
# import nglview

# from collections import OrderedDict
# width = "150px"




## Step 1: Select Structure from Database

In [ ]:
struct_browser = StructureBrowser()
display(struct_browser)

## Step 2: Select a Computer

In [ ]:
# query AiiDA database for Computers
qb = QueryBuilder()
qb.append(Computer, filters={'enabled': True}, project='name', tag='computer')
qb.append(Code, project='*', has_computer='computer', filters={'attributes.input_plugin': 'cp2k'})

all_computers = OrderedDict()
all_computers['Please selecte a computer'] = False
for match in qb.all():
    label = match[0]
    code = match[1]
    all_computers[label] = code

drop_computer = ipw.Dropdown(description="Computer", options=all_computers)
display(drop_computer)

## Step 3: Test submission

In [ ]:
def on_test(b):
    with test_out:
        clear_output()
        calc = build_calc()
        if not calc:
            return
        folder = calc.submit_test()[0]
        fn = folder.get_abs_path(calc._INPUT_FILE_NAME)
        content = open(fn).read()
        pre_tag = '<pre style="width:600px; max-height:250px; overflow-x:auto; line-height:1em; font-size:0.8em;">'   
        box_preview.value = pre_tag+content+"</pre>"
        print("Test went fine :-)")

btn_test = ipw.Button(description="Test")
btn_test.on_click(on_test)
box_preview = ipw.HTML()
test_out = ipw.Output()
display(ipw.VBox([btn_test, box_preview]), test_out)

In [ ]:
def build_calc():
    structure = struct_browser.results.value
    if not structure:
        print("Please select a structure")
        return
    
    code = drop_computer.value
    if not code:
        print("Please select a computer")
        return

    # calc object
    calc = code.new_calc()
    #calc.description = inp_descr.value

    print("This might take a while...")
    # make sure we're really dealing with a gold slab
    atoms = structure.get_ase() # slow
    try:
        first_slab_atom = np.argwhere(atoms.numbers==79)[0,0]
        is_H = atoms.numbers[first_slab_atom:] == 1
        is_Au = atoms.numbers[first_slab_atom:] == 79
        assert np.all(np.logical_or(is_H, is_Au))
        assert np.sum(is_Au) / np.sum(is_H) == 4
    except:
        print("Selected structure is not a proper slab.")
        return
    
    # structure
    molslab_f, mol_f = mk_coord_files(structure, Int(first_slab_atom))
    calc.use_file(molslab_f, linkname='mol_slab_coords')
    calc.use_file(mol_f, linkname='mol_coords')

    # Au potential
    pot_f = SinglefileData(file=path.abspath("Au.pot"))
    calc.use_file(pot_f, linkname='au_pot')
    
    # parameters
    inp = get_cp2k_inp('ABCfoo', 10, 12)
    calc.use_parameters(ParameterData(dict=inp))

    # resources
    calc.set_max_wallclock_seconds(3*60) # 3 min
    calc.set_resources({"num_machines": 1})
    
    return calc

In [ ]:
def mk_coord_files(structure, first_slab_atom):
    first_slab_atom = first_slab_atom.value
    atoms = structure.get_ase() # very slow
    mol = atoms[:first_slab_atom]
    
    tmpdir = tempfile.mkdtemp()
    molslab_fn = tmpdir + 'mol_on_slab.xyz'
    mol_fn = tmpdir + 'mol.xyz'
    
    atoms.write(molslab_fn)
    mol.write(mol_fn)
    
    molslab_f = SinglefileData(file=molslab_fn)
    mol_f = SinglefileData(file=mol_fn)
    
    shutil.rmtree(tmpdir)
    
    return molslab_f, mol_f

In [ ]:
def get_cp2k_inp(cell_abc, first_slab_atom, last_slab_atom):
    inp = {
        'MULTIPLE_FORCE_EVALS': {
            'FORCE_EVAL_ORDER': '2 3',
            'MULTIPLE_SUBSYS': 'T'
        },
        'GLOBAL': {
            'RUN_TYPE': 'GEO_OPT'
        },
        'MOTION': get_motion(first_slab_atom, last_slab_atom),
        'FORCE_EVAL': [force_eval_mixed(cell_abc, first_slab_atom, last_slab_atom),
                       force_eval_fist(cell_abc),
                       get_force_eval_qs(cell_abc),
                      ],
    }
    
    return inp

In [ ]:
def get_motion(first_slab_atom, last_slab_atom):
    motion = {
        'CONSTRAINT': {
            'FIXED_ATOMS': {
                'LIST': '2', 
                ' ': '%d..%d'%(first_slab_atom, last_slab_atom),
            }
        },
        'GEO_OPT': {
            'MAX_FORCE': '1e-3',
            'MAX_ITER': '100'
            
        }
    }
    
    return motion

In [ ]:
def force_eval_mixed(cell_abc, first_slab_atom, last_slab_atom):
    first_mol_atom = 1
    last_mol_atom = first_slab_atom - 1
    
    force_eval = {
        'METHOD': 'MIXED',
        'MIXED': {
            'MIXING_TYPE': 'GENMIX',
            'GROUP_PARTITION': '2 38', ## IN THE HYPOTHESIS OF 40 cores 
            'GENERIC': {
                'ERROR_LIMIT': '1.0E-10',
                'MIXING_FUNCTION': 'E1+E2',
                'VARIABLES': 'E1 E2'
            },
            'MAPPING': {
                'FORCE_EVAL_MIXED': {
                    'FRAGMENT': [{'_': '1', ' ': '%d  %d'%(first_mol_atom, last_mol_atom)},
                                 {'_': '2', ' ': '%d  %d'%(first_slab_atom, last_slab_atom)}],
                    'FORCE_EVAL': [{'_': '1', 'DEFINE_FRAGMENTS': '1 2'},
                                   {'_': '2', 'DEFINE_FRAGMENTS': '1'}],
                
                }
            }
        },
        'SUBSYS': {
            'CELL': {'ABC': cell_abc},
            'TOPOLOGY': {
                'COORD_FILE_NAME': 'mol_on_slab.xyz',
                'COORDINATE': 'XYZ',
                'CONNECTIVITY': 'OFF',
            }
        }
    }
    
    return force_eval

In [ ]:
def force_eval_fist(cell_abc):
    ff = {
            'SPLINE': {
                'EPS_SPLINE': '1.30E-5',
                'EMAX_SPLINE': '0.8',
            },
            'CHARGE' : [],
            'NONBONDED': {
                'GENPOT': [],
                'LENNARD-JONES': [],
                'EAM':{
                   'ATOMS': 'Au Au',
                   'PARM_FILE_NAME': 'Au.pot', 
                },
            },
    }
    
    for x in ('Au', 'H', 'C', 'O', 'N'):
        ff['CHARGE'].append({'ATOM':x, 'CHARGE': 0.0})
        
    for x in ('C', 'N', 'O', 'H'):
        ff['NONBONDED']['GENPOT'].append(
            {'ATOMS': 'Au ' + x,
             'FUNCTION': 'A*exp(-av*r)+B*exp(-ac*r)-C/(r^6)/( 1+exp(-20*(r/R-1)) )',
             'VARIABLES': 'r',
             'PARAMETERS': 'A av B ac C R',
             'VALUES': '4.13643 1.33747 115.82004 2.206825 113.96850410723008483218 5.84114',
             'RCUT': '15'}
        )

    for x in ('C H', 'H H', 'H N', 'C C', 'C O', 'C N', 'N N', 'O H', 'O N', 'O O'):
        ff['NONBONDED']['LENNARD-JONES'].append(
            {'ATOMS': x,
             'EPSILON': '0.0',
             'SIGMA': '3.166',
             'RCUT': '15'}
        )
        
    force_eval = {
        'METHOD': 'FIST',
        'MM':{
            'FORCEFIELD': ff,
            'POISSON': {
                'EWALD': {
                   'EWALD_TYPE': 'none',
                },
            },
        },
        'SUBSYS': {
            'CELL': {
              'ABC': cell_abc,
            },
            'TOPOLOGY':{
              'COORD_FILE_NAME': 'mol_on_slab.xyz',
              'COORDINATE': 'XYZ',
              'CONNECTIVITY': 'OFF',
            },
        },
    }
    return force_eval

In [ ]:
def get_force_eval_qs(cell_abc):
    force_eval = {
        'METHOD': 'Quickstep',
        'DFT': {
            'QS': {
                'METHOD': 'DFTB',
                'EXTRAPOLATION': 'ASPC',
                'EXTRAPOLATION_ORDER': '3',
                'DFTB': {
                    'SELF_CONSISTENT': 'T',
                    'DISPERSION': 'T',
                    'ORTHOGONAL_BASIS': 'F',
                    'DO_EWALD': 'F',
                },
                'PARAMETER': {
                    'PARAM_FILE_PATH': 'DFTB/scc',
                    'PARAM_FILE_NAME': 'scc_parameter',
                    'UFF_FORCE_FIELD': '../uff_table',
                }
            },
            'SCF': {    
                'MAX_SCF': '30',
                'SCF_GUESS': 'RESTART',
                'EPS_SCF': '1.0E-6',
                'OT': {
                    'PRECONDITIONER': 'FULL_SINGLE_INVERSE',
                    'MINIMIZER': 'CG',
                },
                'OUTER_SCF': {
                    'MAX_SCF': '20',
                    'EPS_SCF': '1.0E-6',
                },
                'PRINT': {
                    'RESTART': {
                        'EACH': {
                            'QS_SCF': '0',
                            'GEO_OPT': '1',
                        },
                        'ADD_LAST': 'NUMERIC',
                        'FILENAME': 'RESTART'
                    },
                    'RESTART_HISTORY': {'_': 'OFF'}
                }
            }
        },
        'SUBSYS': {
            'CELL': {'ABC': cell_abc},
            'TOPOLOGY': {
                'COORD_FILE_NAME': 'mol.xyz',
                'COORDINATE': 'xyz'
            }
        }
    }
  
    return force_eval

## Step 4: Submit calculation

In [ ]:
def on_submit(b):
    with submit_out:
        clear_output()
        calc = build_calc()
        
        # store and submit
        calc.store_all()
        job = calc.submit()
        print(job)

btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)